In [1]:
# import packages
import numpy as np
import pandas as pd
import calendar
from tqdm.notebook import trange, tqdm
from math import radians, cos, sin, asin, sqrt 

# suppress warning
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
dataset = pd.read_csv("201901-citibike-tripdata.csv")
# dataset.head()

In [3]:
# Havensine Greater-circle distance function
def distance(lat1, lon1, lat2, lon2): 
    lat1 = radians(lat1) 
    lon1 = radians(lon1) 
    lat2 = radians(lat2)
    lon2 = radians(lon2) 
     
    dlon = lon2 - lon1  
    dlat = lat2 - lat1 
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * asin(sqrt(a))  
    r = 6371                # Earth radius r = 3956 miles/6371 km
    return(c * r)

In [4]:
# Initializing dataframes to be appended & exported
monthly_agg_dataset = pd.DataFrame(columns= ["year", "month", "avg_distance", "avg_trip_duration"])
final_dataset_collated  = pd.DataFrame(columns = ["year", "month", "hour", "avg_concurrent_riders", "avg_distance", "avg_trip_duration"])
chronological_dataset = pd.DataFrame(columns = ["year", "month", "day", "hour", "concurrent_riders", "avg_distance", "avg_trip_duration"]) 

# Defining time parameters
yl = 2019
yu = 2020 + 1
ml = 1
mu = 12 + 1

# Core Sub-routine: NOTE - This sub-routine doesn't account for concurrent riders/trips overlapping through a month end to the next
for y in range(yl,yu):
    for m in tqdm(range(ml,mu), desc=str(y)+" Month Loop"):

        # reading csv file
        filename = str(y)+str(m).zfill(2)+"-citibike-tripdata.csv"
        dataset = pd.read_csv(filename)
        
        # computing distance with Havensine formula
        dataset["distance"] = dataset.apply(lambda row: distance(row["start station latitude"], row["start station longitude"], row["end station latitude"], row["end station longitude"]), axis=1)
        
        # converting timestamps from str to datetime format
        # dataset["tripduration"] = dataset["tripduration"]
        dataset["starttime2"] = pd.to_datetime(dataset['starttime'])
        dataset["stoptime2"] = pd.to_datetime(dataset['stoptime'])
        
        # truncating microseconds
        dataset["starttime2"] = dataset.apply(lambda row: row["starttime2"].replace(microsecond=0), axis=1)
        dataset["stoptime2"] = dataset.apply(lambda row: row["stoptime2"].replace(microsecond=0), axis=1)
        
        # selecting required columns
        dataset = dataset[["tripduration", "distance", "starttime2", "stoptime2", "start station latitude", "start station longitude", "end station latitude", "end station longitude"]]
        
        # initializing intermediate aggregating dataframe that will be appended to the dataframes initiated at the top
        agg_dataset = pd.DataFrame(columns = ["year", "month", "day", "hour", "concurrent_riders", "distance", "trip_duration", "temp"])
        
        # get the number of days in that month and execute a for loop
        for d in range(1,calendar.monthrange(y, m)[1]+1):
            for h in range(0,24):
                
                # defining lower & upper bound hour for computing concurrent riders/trips
                lowerbound = pd.Timestamp(year=y, month=m, day=d, hour=h)
                upperbound = lowerbound + pd.to_timedelta(1, unit='h')
                
                # dataset2 filters out ongoing concurrent riders/trips in the lowerbound-upperbound duration defined above
                dataset2 = dataset[(dataset.starttime2 < upperbound) & ((dataset.stoptime2 > lowerbound) & (dataset.stoptime2 > dataset.starttime2))]
                # dataset3 filters out trips that began in the lowerbound-upperbound duration defined above
                dataset3 = dataset[(dataset.starttime2 >= lowerbound) & (dataset.starttime2 < upperbound)]
                
                concurrent_riders = dataset2.shape[0]
                # temp keeps a counter of trips used for computing avg below after aggregation through [year,month,hour]
                temp = dataset3.shape[0]
                
                # computing total trip duration and converting it to minutes
                total_trip_duration = dataset3["tripduration"].sum()/60
                # computing total distance
                total_distance = dataset3["distance"].sum()
                
                # appending above computed variables into the aggregation dataset
                agg_dataset = agg_dataset.append({"year": y, "month": m, "day": d, "hour": h, "concurrent_riders": concurrent_riders, "distance":total_distance, "trip_duration":total_trip_duration, "temp":temp}, ignore_index = True)
        
        # computing below parameters for a month-by-month basis
        monthly_avg_distance = dataset["distance"].sum()/dataset.shape[0]
        monthly_avg_trip_duration = dataset["tripduration"].sum()/dataset.shape[0]
        # converting to minutes
        monthly_avg_trip_duration /= 60
        # appending above computed variables to the monthly aggregation dataset
        monthly_agg_dataset = monthly_agg_dataset.append({"year": y, "month": m, "avg_distance": monthly_avg_distance, "avg_trip_duration": monthly_avg_trip_duration}, ignore_index = True)
        
        # converting aggregation dataset variables to appropriate datatypes
        agg_dataset = agg_dataset.astype({"year":int, "month":int, "day":int, "hour":int, "concurrent_riders":int, "temp":int})
        
        # copying aggregation dataset which is then grouped by [year, month, hour]
        final_dataset= agg_dataset[["year", "month", "hour", "concurrent_riders", "distance", "trip_duration", "temp"]]
        final_dataset = final_dataset.groupby(["year","month","hour"], as_index = False).sum()
        # calculating averages
        final_dataset["concurrent_riders"] = final_dataset["concurrent_riders"]/calendar.monthrange(y, m)[1]
        final_dataset["distance"] = final_dataset["distance"]/final_dataset["temp"]
        final_dataset["trip_duration"] = final_dataset["trip_duration"]/final_dataset["temp"]
        # dropping temp column as it is no longer required
        final_dataset.drop(["temp"], axis = 1, inplace = True)
        # renaming columns after computing averages
        final_dataset.rename(columns = {"concurrent_riders": "avg_concurrent_riders", "distance": "avg_distance", "trip_duration": "avg_trip_duration"}, inplace = True) 
        
        # original aggregation dataset without any grouping to retain chronological granularity
        # calculating averages
        agg_dataset["distance"] = agg_dataset["distance"]/temp
        agg_dataset["trip_duration"] = agg_dataset["trip_duration"]/temp
        # dropping temp column as it is no longer required
        agg_dataset.drop(["temp"], axis = 1, inplace = True)
        # renaming columns after computing averages
        agg_dataset.rename(columns = {"distance": "avg_distance", "trip_duration": "avg_trip_duration"}, inplace = True)
        
        # appending to the intermediate dataframes to the final dataframes to be exported as csv
        chronological_dataset = chronological_dataset.append(agg_dataset, ignore_index = True)
        final_dataset_collated = final_dataset_collated.append(final_dataset, ignore_index = True)

In [5]:
# Exporting dataframes as csv
monthly_agg_dataset.to_csv("monthly_agg_dataset.csv", index = False)
chronological_dataset.to_csv("chronological_dataset.csv", index = False)
final_dataset_collated.to_csv("final_dataset_collated.csv", index = False)